# Abstract

In this competition, Santander Group is asking Kagglers to help them identify the value of transactions for each potential customer. This is a first step that Santander needs to nail in order to personalize their services at scale.
The evaluation metric for this competition is Root Mean Squared Logarithmic Error.

Submission File: For every row in the test.csv, submission files should contain two columns: ID and target. The ID corresponds to the column of that ID in the test.csv.
For the prediction of the Target column used LightGBM and XGboost and at the end combined the predictions of both the libraries and generated the sub_lgb_xgb.csv file as the output.

Importing the libraries of pandas and lightgbm. Loading the train.csv and test.csv files.

In [ ]:
import pandas as pd
import lightgbm as lgb
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

Retreiving the first five rows of train dataframe

![](http://)

In [ ]:
train.head()

Retreiving the first five rows of test dataframe

In [ ]:
test.head()

Removing the NaN values from the loades data in the train dataframe

In [ ]:
print("Total Train Features with NaN Values = " + str(train.columns[train.isnull().sum() != 0].size))
if (train.columns[train.isnull().sum() != 0].size):
    print("Features with NaN => {}".format(list(train.columns[train.isnull().sum() != 0])))
    train[train.columns[train.isnull().sum() != 0]].isnull().sum().sort_values(ascending = False)

Removing the NaN values from the loades data in the test dataframe

In [ ]:
print("Total Test Features with NaN Values = " + str(test.columns[test.isnull().sum() != 0].size))
if (test.columns[test.isnull().sum() != 0].size):
    print("Features with NaN => {}".format(list(test.columns[test.isnull().sum() != 0])))
    test[test.columns[test.isnull().sum() != 0]].isnull().sum().sort_values(ascending = False)

Importing numpy library and dropping target column.

In [ ]:
import numpy as np
X_train = train.drop(["ID", "target"], axis=1)
y_train = np.log1p(train["target"].values)

X_test = test.drop(["ID"], axis=1)

Removing constant columns in the training set and test set.

In [ ]:
colsToRemove = []
for col in X_train.columns:
    if X_train[col].std() == 0: 
        colsToRemove.append(col)
        
# remove constant columns in the training set
X_train.drop(colsToRemove, axis=1, inplace=True)

# remove constant columns in the test set
X_test.drop(colsToRemove, axis=1, inplace=True) 

print("Removed `{}` Constant Columns\n".format(len(colsToRemove)))
print(colsToRemove)

Removing the duplicate columns in the training set and test set.

In [ ]:
colsToRemove = []
colsScaned = []
dupList = {}

columns = X_train.columns

for i in range(len(columns)-1):
    v = X_train[columns[i]].values
    dupCols = []
    for j in range(i+1,len(columns)):
        if np.array_equal(v, X_train[columns[j]].values):
            colsToRemove.append(columns[j])
            if columns[j] not in colsScaned:
                dupCols.append(columns[j]) 
                colsScaned.append(columns[j])
                dupList[columns[i]] = dupCols
                
# remove duplicate columns in the training set
X_train.drop(colsToRemove, axis=1, inplace=True) 

# remove duplicate columns in the testing set
X_test.drop(colsToRemove, axis=1, inplace=True)

print("Removed `{}` Duplicate Columns\n".format(len(dupList)))
print(dupList)

In [ ]:
# from sklearn.ensemble import GradientBoostingRegressor
# clf_gb = GradientBoostingRegressor(random_state = 42)
# clf_gb.fit(X_train, y_train)
# print(clf_gb)

In [ ]:
import matplotlib.pyplot as plt
feat_importances = pd.Series(clf_gb.feature_importances_, index=X_train.columns)
feat_importances = feat_importances.nlargest(100)
plt.figure(figsize=(16,15))
feat_importances.plot(kind='barh')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
# feat_importances_gb = pd.Series(clf_gb.feature_importances_, index=X_train.columns)
# feat_importances_gb = feat_importances_gb.nlargest(25)
# plt.figure(figsize=(16,8))
# feat_importances_gb.plot(kind='barh')
# plt.gca().invert_yaxis()
# plt.show()

In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# clf_rf = RandomForestRegressor(random_state = 42)
# clf_rf.fit(X_train, y_train)
# print(clf_rf)

In [ ]:
# def evaluate(model, features, labels):
#     predictions = model.predict(features)
#     errors = abs(predictions - labels)
#     m = 100 * np.mean(errors / labels)
#     accuracy = 100 - m
#     print('Model Performance')
#     print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
#     print('Accuracy = {:0.2f}%.'.format(accuracy))
    
#     return accuracy

In [ ]:
# from sklearn.preprocessing import OneHotEncoder
# onehotencoder = OneHotEncoder()
# for column in train.columns:
#     train[column] = onehotencoder.fit_transform(train[column])
#     train[column] = train[column].reshape(1, -1)
#     print(train[column])
    
    
    
#     #print test       # Produces: [ 0.58  0.76]
# #print test.shape # Produces: (2,) meaning 2 rows, 1 col

# #test = test.reshape(1, -1)
# #print test       # Produces: [[ 0.58  0.76]]
# #print test.shape 


In [ ]:
# RF_target=np.expm1(clf_rf.predict(test))

In [ ]:
feat_importances_rf = pd.Series(clf_rf.feature_importances_, index=X_train.columns)
feat_importances_rf = feat_importances_rf.nlargest(25)
plt.figure(figsize=(16,8))
feat_importances_rf.plot(kind='barh')
plt.gca().invert_yaxis()
plt.show()

Impoting library and generating pairplot of Top 6 Important features.

In [ ]:
import seaborn as sns
df_plot = X_train[['f190486d6', 'eeb9cd3aa', '58e2e02e6', '58232a6fb', '15ace8c9f', '9fd594eec']]
df_plot['target'] = y_train

g = sns.pairplot(df_plot, diag_kind="kde", palette="BuGn_r")
g.fig.suptitle('Pairplot of Top 6 Important Features',fontsize=26)

In [ ]:
s1 = pd.Series(clf_gb.feature_importances_, index=X_train.columns).nlargest(20).index
s2 = pd.Series(clf_rf.feature_importances_, index=X_train.columns).nlargest(20).index

common_features = pd.Series(list(set(s1).union(set(s2)))).values

print(common_features)

Generating correlation Heatmap.

In [ ]:
df_plot = pd.DataFrame(X_train, columns = common_features)
corr = df_plot.corr()

mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(16, 16))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.title("Correlation HeatMap", fontsize=15)
plt.show()

Build Test and Train data for modeling.

In [ ]:
from sklearn.model_selection import train_test_split
dev_X, val_X, dev_y, val_y = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)

LightGBM for predicting the target value. Light GBM is a gradient boosting framework that uses tree based learning algorithm. Light GBM grows tree vertically while other algorithm grows trees horizontally meaning that Light GBM grows tree leaf-wise while other algorithm grows level-wise.

In [ ]:
from sklearn import model_selection
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import mean_squared_error
def run_lgb(train_X, train_y, val_X, val_y, test_X):
    params = {
        "objective" : "regression",
        "metric" : "rmse",
        "num_leaves" : 35,
        "learning_rate" : 0.005,
        "bagging_fraction" : 0.7,
        "feature_fraction" : 0.5,
        "bagging_frequency" : 5,
        "bagging_seed" : 42,
        "verbosity" : -1,
        "seed": 42
    }
    
    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    evals_result = {}
    model = lgb.train(params, lgtrain, 5000, 
                      valid_sets=[lgval], 
                      early_stopping_rounds=500, 
                      verbose_eval=50, 
                      evals_result=evals_result)
    
    pred_test_y = np.expm1(model.predict(test_X, num_iteration=model.best_iteration))
    return pred_test_y, model, evals_result

In [ ]:
# pred_test, model, evals_result = run_lgb(dev_X, dev_y, val_X, val_y, X_test)
# print("LightGBM Training Completed...")

In [ ]:
# # feature importance
# print("Features Importance...")
# gain = model.feature_importance('gain')
# featureimp = pd.DataFrame({'feature':model.feature_name(), 
#                    'split':model.feature_importance('split'), 
#                    'gain':100 * gain / gain.sum()}).sort_values('gain', ascending=False)
# print(featureimp[:15])

In [ ]:
# def run_xgb(train_X, train_y, val_X, val_y, test_X):
#     print("############# Build XGBoost model #############")
#     model_xgb = xgb.XGBRegressor(colsample_bytree = 0.055, 
#                                  colsample_bylevel = 0.5, 
#                                  gamma = 1.5, 
#                                  learning_rate = 0.02, 
#                                  max_depth = 32, 
#                                  objective = 'reg:linear',
#                                  booster = 'gbtree',
#                                  min_child_weight = 57, 
#                                  n_estimators = 1000, 
#                                  reg_alpha = 0, 
#                                  reg_lambda = 0,
#                                  eval_metric = 'rmse', 
#                                  subsample = 0.7, 
#                                  silent = 1, 
#                                  n_jobs = -1, 
#                                  early_stopping_rounds = 14,
#                                  random_state = 7, 
#                                  nthread = -1)
    
#     print("Validating.....")
#     model_xgb.fit(x_train, y_train, eval_set = [(x_train, y_train), (x_val, y_val)], eval_metric = 'rmse', 
#                   early_stopping_rounds = 100, verbose = True)
#     pred_test_y = np.expm1(model_xgb.predict(test_X))
#     print("XGBoost Training Completed...")
    
#     return pred_test_y
    

In [ ]:
pred_test, model, evals_result = run_lgb(dev_X, dev_y, val_X, val_y, X_test)
print("LightGBM Training Completed...")

In [ ]:
print("Features Importance...")
gain = model.feature_importance('gain')
featureimp = pd.DataFrame({'feature':model.feature_name(), 
                   'split':model.feature_importance('split'), 
                   'gain':100 * gain / gain.sum()}).sort_values('gain', ascending=False)
print(featureimp[:15])

In [ ]:
# sub=pd.read_csv('../input/sample_submission.csv')

# sub_rf = pd.DataFrame()
# sub_rf["target"] = RF_target
# sub_rf["ID"] = sub["ID"]
# sub_rf.to_csv("sub_rf.csv", index=False)

# sub_lgb = pd.DataFrame()
# sub_lgb["target"] = lgb_pred
# sub_lgb["ID"] = sub["ID"]
# sub_lgb.to_csv("sub_lgb.csv", index=False)


# sub["target"] = (sub_lgb["target"] + sub_rf['target'] )/2

# print(sub.head())
# sub.to_csv('sub_lgb_xgb.csv', index=False)
# sub.head()

In [ ]:
# # feature importance
# print("Features Importance...")
# gain = model.feature_importance('gain')
# featureimp = pd.DataFrame({'feature':model.feature_name(), 
#                    'split':model.feature_importance('split'), 
#                    'gain':100 * gain / gain.sum()}).sort_values('gain', ascending=False)
# print(featureimp[:15])

In [ ]:
# def run_xgb(dev_X, dev_y, val_X, val_y, X_test):
#     params = {'objective': 'reg:linear', 
#           'eval_metric': 'rmse',
#           'eta': 0.001,
#           'max_depth': 10, 
#           'subsample': 0.6, 
#           'colsample_bytree': 0.6,
#           'alpha':0.001,
#           'random_state': 42, 
#           'silent': True}
    
#     tr_data = xgb.DMatrix(train_X, train_y)
#     va_data = xgb.DMatrix(val_X, val_y)
    
#     watchlist = [(tr_data, 'train'), (va_data, 'valid')]
    
#     model_xgb = xgb.train(params, tr_data, 2000, watchlist, maximize=False, early_stopping_rounds = 100, verbose_eval=100)
    
#     dtest = xgb.DMatrix(test_X)
#     xgb_pred_y = np.expm1(model_xgb.predict(dtest, ntree_limit=model_xgb.best_ntree_limit))
    
#     return xgb_pred_y, model_xgb

In [ ]:
# pred_test, model, evals_result = run_xgb(dev_X, dev_y, val_X, val_y, X_test)
# print("xgboost Training Completed...")

XGB modeling to find the Target column. XGBoost is an optimized distributed gradient boosting library designed to be highly efficient, flexible and portable. It implements machine learning algorithms under the Gradient Boosting framework. XGBoost provides a parallel tree boosting (also known as GBDT, GBM) that solve many data science problems in a fast and accurate way. 

In [ ]:
import xgboost as xgb
def run_xgb(train_X, train_y, val_X, val_y, test_X):
    params = {'objective': 'reg:linear', 
          'eval_metric': 'rmse',
          'eta': 0.005,
          'max_depth': 15, 
          'subsample': 0.7, 
          'colsample_bytree': 0.5,
          'alpha':0,
          'random_state': 42, 
          'silent': True}
    
    tr_data = xgb.DMatrix(train_X, train_y)
    va_data = xgb.DMatrix(val_X, val_y)
    
    watchlist = [(tr_data, 'train'), (va_data, 'valid')]
    
    model_xgb = xgb.train(params, tr_data, 2000, watchlist, maximize=False, early_stopping_rounds = 30, verbose_eval=100)
    
    dtest = xgb.DMatrix(test_X)
    xgb_pred_y = np.expm1(model_xgb.predict(dtest, ntree_limit=model_xgb.best_ntree_limit))
    
    return xgb_pred_y, model_xgb


In [ ]:
# Training XGB
pred_test_xgb, model_xgb = run_xgb(dev_X, dev_y, val_X, val_y, X_test)
print("XGB Training Completed...")

In [ ]:
# 

In [ ]:
# sub = pd.read_csv('../input/sample_submission.csv')
# sub["target"] = pred_test
# print(sub.head())
# sub.to_csv('sub_lgb.csv', index=False)

Combining predictions from both LightGBM and xgBoost model.

In [ ]:
sub = pd.read_csv('../input/sample_submission.csv')

sub_lgb = pd.DataFrame()
sub_lgb["target"] = pred_test

sub_xgb = pd.DataFrame()
sub_xgb["target"] = pred_test_xgb

sub["target"] = (sub_lgb["target"] + sub_xgb["target"])/2

Generating .CSV file for submission.

In [ ]:
print(sub.head())
sub.to_csv('sub_lgb_xgb.csv', index=False)

**CONCLUSION**

From the above observations we can infer that:

###### The XGboost classifier has a Mean F-Score of 0.344
######  

As F-Score value is a measure of a test's accuracy, hence a model with high Mean F-score is preferred. 
From the above results, we prefer AdaBoost classifier as it has maximum F-Score value

**CONTRIBUTION STATEMENT**

60% of code from online resources 40% of code by me.

**CITATIONS**

Kaggle: https://www.kaggle.com/ 
Wikipedia: https://en.wikipedia.org/ 
StackOverflow: https://stackoverflow.com/
Github: https://github.com/nikbearbrown/INFO_6105
classroom.github.com/assignment-invitations/a1008a519017ca80da27fb845ab72818/success
Kaggle Kernel: https://www.kaggle.com/aditya1702/adversarial-val-lgbm-xgb-model-lb-1-40
https://www.kaggle.com/samratp/lightgbm-xgboost-catboost

**LICENSE**

Copyright 2019 FNU KARAN
Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions: The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software. THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE